In [11]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from inxss import SirenNet

import glob, os

from sklearn.model_selection import train_test_split

from lightning.pytorch.loggers import TensorBoardLogger

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from inxss import SpectrumDataset, SpecNeuralRepr
from inxss.dataset import FullSpectrumDataset
from inxss.specnet import FullSpectrumNetwork
from torch.utils.data import DataLoader

In [13]:
mmapped_data_filename = '/pscratch/sd/z/zhantao/inxs_steering/smoothed_SpinW_data/memmap_datasets/S_sigma_0.75.dat'
S_shape = torch.load('/pscratch/sd/z/zhantao/inxs_steering/smoothed_SpinW_data/memmap_datasets/S_shape.pt')['S_shape']
mmapped_data = np.memmap(mmapped_data_filename, dtype=np.float32, mode='r', shape=S_shape)

In [14]:
data_folder = '/pscratch/sd/z/zhantao/inxs_steering/smoothed_SpinW_data/sigma_0.75'

data_file = sorted(glob.glob(os.path.join(data_folder, '*.pt')))

train_data_file, val_test_data_file = train_test_split(data_file, test_size=1/6, random_state=42)
val_data_file,   test_data_file     = train_test_split(val_test_data_file, test_size=0.5, random_state=42)

train_dataset = FullSpectrumDataset(
    data_file = train_data_file,
    grid_file = '/pscratch/sd/z/zhantao/inxs_steering/smoothed_SpinW_data/sigma_0.75/grid_metadata',
    num_coords_per_sample = 10000
)
val_dataset = FullSpectrumDataset(
    data_file = val_data_file,
    grid_file = '/pscratch/sd/z/zhantao/inxs_steering/smoothed_SpinW_data/sigma_0.75/grid_metadata',
    num_coords_per_sample = 10000
)
test_dataset = FullSpectrumDataset(
    data_file = test_data_file,
    grid_file = '/pscratch/sd/z/zhantao/inxs_steering/smoothed_SpinW_data/sigma_0.75/grid_metadata',
    num_coords_per_sample = 10000
)

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True, num_workers=10)
val_loader = DataLoader(val_dataset, batch_size=10, shuffle=False, num_workers=10)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False, num_workers=10)

In [15]:
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint, TQDMProgressBar

model = FullSpectrumNetwork(
    scale_dict={
        'h' : [(0, 1), (0, 10)],
        'k' : [(0, 1), (0, 10)],
        'w' : [(0, 150), (0, 15)]
        }
    )

In [16]:
checkpoint_callback = ModelCheckpoint(
    save_on_train_epoch_end=False, save_last=True, save_top_k=1, monitor="val_loss"
)

logger = TensorBoardLogger(save_dir='/pscratch/sd/z/zhantao/inxs_steering')

trainer = L.Trainer(
    max_epochs=1000, accelerator="gpu",
    callbacks=[TQDMProgressBar(refresh_rate=10), checkpoint_callback],
    logger=logger, log_every_n_steps=1, devices=1,
    enable_checkpointing=True,
    default_root_dir='/pscratch/sd/z/zhantao/inxs_steering'
)

trainer.fit(model, train_loader, val_loader)

/pscratch/sd/z/zhantao/conda/inxs/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /pscratch/sd/z/zhantao/conda/inxs/lib/python3.9/site ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/pscratch/sd/z/zhantao/conda/inxs/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /pscratch/sd/z/zhantao/conda/inxs/lib/python3.9/site ...
  rank_zero_warn(
You are using a CUDA device ('NVIDIA A100-PCIE-40GB') that has Tensor 

Epoch 18:  40%|████      | 20/50 [00:10<00:15,  1.92it/s, v_num=1.49e+7]   

/pscratch/sd/z/zhantao/conda/inxs/lib/python3.9/site-packages/lightning/pytorch/trainer/call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


: 